<a href="https://colab.research.google.com/github/galenzo17/AI-personal-test/blob/main/risk_model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Paso 1: Instalar y actualizar Gradio
!pip install --upgrade gradio

# Paso 2: Importar bibliotecas necesarias
import gradio as gr
import joblib
import numpy as np
import os
from google.colab import drive

# Importar bibliotecas para gráficos
import matplotlib.pyplot as plt
import io
import base64

# Paso 3: Montar Google Drive (si aún no está montado)
drive.mount('/content/drive')

# Definir las rutas a tus archivos
modelo_path = '/content/drive/MyDrive/Modelos/modelo_riesgo_crediticio.pkl'  # Ajusta la ruta según corresponda
scaler_path = '/content/drive/MyDrive/Modelos/scaler.pkl'  # Ajusta la ruta según corresponda

# Verificar que los archivos existen
if not os.path.exists(modelo_path):
    print(f"El archivo del modelo no se encontró en la ruta: {modelo_path}")
if not os.path.exists(scaler_path):
    print(f"El archivo del scaler no se encontró en la ruta: {scaler_path}")

# Cargar el modelo usando joblib
try:
    modelo = joblib.load(modelo_path)
    print("Modelo cargado correctamente.")
except Exception as e:
    print(f"Error al cargar el modelo: {e}")

# Cargar el scaler usando joblib
try:
    scaler = joblib.load(scaler_path)
    print("Scaler cargado correctamente.")
except Exception as e:
    print(f"Error al cargar el scaler: {e}")

# Definir la función de predicción
def predecir_riesgo_crediticio(
    age,
    income,
    debt,
    person_home_ownership,
    loan_intent,
    loan_grade,
    cb_person_default_on_file,
    employment_length,
    annual_percentage_rate,
    verification_status,
    purpose
):
    # Mapeo de las variables categóricas a sus valores numéricos
    home_ownership_mapping = {'OWN': 0, 'MORTGAGE': 1, 'RENT': 2}
    loan_intent_mapping = {
        'debt_consolidation': 0,
        'credit_card': 1,
        'home_improvement': 2,
        'major_purchase': 3,
        'small_business': 4,
        'car': 5,
        'medical': 6
    }
    loan_grade_mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6}
    default_on_file_mapping = {'Yes': 1, 'No': 0}
    verification_status_mapping = {'Verified': 0, 'Source Verified': 1, 'Not Verified': 2}
    purpose_mapping = {'credit_card': 0, 'debt_consolidation': 1, 'home_improvement': 2, 'major_purchase': 3, 'small_business': 4, 'car': 5, 'medical': 6, 'vacation': 7, 'moving': 8, 'house': 9}

    # Codificación de variables categóricas
    person_home_ownership_encoded = home_ownership_mapping.get(person_home_ownership, 1)  # Default a 'MORTGAGE' si no coincide
    loan_intent_encoded = loan_intent_mapping.get(loan_intent, 1)  # Default a 'credit_card' si no coincide
    loan_grade_encoded = loan_grade_mapping.get(loan_grade, 3)  # Default a 'D' si no coincide
    cb_person_default_on_file_encoded = default_on_file_mapping.get(cb_person_default_on_file, 0)  # Default a 'No' si no coincide
    verification_status_encoded = verification_status_mapping.get(verification_status, 2)  # Default a 'Not Verified' si no coincide
    purpose_encoded = purpose_mapping.get(purpose, 0)  # Default a 'credit_card' si no coincide

    # Validaciones
    if age < 18 or age > 100:
        return "Edad inválida. Por favor, ingresa una edad entre 18 y 100 años."
    if income < 0:
        return "Ingresos inválidos. Por favor, ingresa un valor positivo."
    if debt < 0:
        return "Deuda inválida. Por favor, ingresa un valor positivo."
    if employment_length < 0:
        return "Años de empleo inválidos. Por favor, ingresa un valor positivo."
    if annual_percentage_rate < 0 or annual_percentage_rate > 100:
        return "Tasa de porcentaje anual inválida. Por favor, ingresa un valor entre 0 y 100."

    # Crear el arreglo de características en el mismo orden que durante el entrenamiento
    datos = np.array([
        age,
        income,
        debt,
        person_home_ownership_encoded,
        loan_intent_encoded,
        loan_grade_encoded,
        cb_person_default_on_file_encoded,
        employment_length,
        annual_percentage_rate,
        verification_status_encoded,
        purpose_encoded
    ]).reshape(1, -1)

    # Aplicar el escalador
    try:
        datos_escalados = scaler.transform(datos)
    except Exception as e:
        return f"Error al escalar los datos: {e}"

    # Realizar la predicción
    try:
        probabilidades = modelo.predict_proba(datos_escalados)[0]
        probabilidad_incumplimiento = probabilidades[1]
    except Exception as e:
        return f"Error al realizar la predicción: {e}"

    # Determinar el nivel de riesgo basado en la probabilidad
    if probabilidad_incumplimiento >= 0.8:
        riesgo = "Muy Alto Riesgo"
        comentario = "Existe una alta probabilidad de incumplimiento. Se recomienda una revisión detallada del crédito."
    elif 0.6 <= probabilidad_incumplimiento < 0.8:
        riesgo = "Alto Riesgo"
        comentario = "Existe una probabilidad considerable de incumplimiento. Se recomienda cautela en la aprobación del crédito."
    elif 0.4 <= probabilidad_incumplimiento < 0.6:
        riesgo = "Riesgo Moderado"
        comentario = "Existe una probabilidad moderada de incumplimiento. Se sugiere evaluar otros factores antes de la aprobación."
    else:
        riesgo = "Bajo Riesgo"
        comentario = "Existe una baja probabilidad de incumplimiento. Es probable que el crédito sea aprobado."

    # Crear gráfico de probabilidades
    labels = ['Cumplimiento', 'Incumplimiento']
    probabilities = [probabilidades[0], probabilidad_incumplimiento]

    plt.figure(figsize=(4, 3))
    bars = plt.bar(labels, probabilities, color=['green', 'red'])
    plt.ylim(0, 1)
    plt.title('Probabilidades de Riesgo')
    plt.ylabel('Probabilidad')

    # Añadir etiquetas de probabilidad encima de las barras
    for bar, prob in zip(bars, probabilities):
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2.0, yval + 0.02, f'{prob:.2f}', ha='center', va='bottom')

    # Guardar el gráfico en un buffer
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close()
    buf.seek(0)
    img_base64 = base64.b64encode(buf.read()).decode()

    # Formatear el resultado con el gráfico
    resultado = (
        f"### **{riesgo}**\n"
        f"**Probabilidad de Incumplimiento:** {probabilidad_incumplimiento:.2f}\n\n"
        f"**Comentario:** {comentario}\n\n"
        f"![Probabilidades](data:image/png;base64,{img_base64})"
    )

    return resultado

# Definir la interfaz de Gradio usando la nueva sintaxis
iface = gr.Interface(
    fn=predecir_riesgo_crediticio,
    inputs=[
        gr.Number(label="Edad"),
        gr.Number(label="Ingresos Anuales (income)"),
        gr.Number(label="Deuda Actual (debt)"),
        gr.Dropdown(choices=["OWN", "MORTGAGE", "RENT"], label="Propiedad de la Vivienda (person_home_ownership)"),
        gr.Dropdown(
            choices=["debt_consolidation", "credit_card", "home_improvement", "major_purchase", "small_business", "car", "medical"],
            label="Intención del Préstamo (loan_intent)"
        ),
        gr.Dropdown(choices=["A", "B", "C", "D", "E", "F", "G"], label="Calificación del Préstamo (loan_grade)"),
        gr.Dropdown(choices=["Yes", "No"], label="Default en Archivo (cb_person_default_on_file)"),
        gr.Number(label="Años de Empleo (employment_length)"),
        gr.Number(label="Tasa de Porcentaje Anual (annual_percentage_rate)"),
        gr.Dropdown(choices=["Verified", "Source Verified", "Not Verified"], label="Estado de Verificación (verification_status)"),
        gr.Dropdown(choices=["credit_card", "debt_consolidation", "home_improvement", "major_purchase", "small_business", "car", "medical", "vacation", "moving", "house"], label="Propósito del Préstamo (purpose)")
    ],
    outputs=gr.Markdown(label="Resultado"),
    title="Análisis de Riesgo Crediticio",
    description="Introduce los datos para predecir el riesgo crediticio."
)

# Lanzar la interfaz de Gradio
iface.launch(share=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Modelo cargado correctamente.
Scaler cargado correctamente.
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ae2d9fcaa83897cc4a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
